In [2]:
# %% [Install Dependencies]
%pip install transformers datasets torch sentencepiece accelerate pandas scikit-learn tldextract ipwhois -q

# %% [Import Libraries]
import pandas as pd
import numpy as np
import torch
import re
import ipaddress
import tldextract
from urllib.parse import urlparse
from sklearn.model_selection import train_test_split
from datasets import Dataset, load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    pipeline
)
from ipwhois import IPWhois
from datetime import datetime

# %% [Load Dataset]
dataset = load_dataset("ealvaradob/phishing-dataset", "urls", trust_remote_code=True)

df = pd.DataFrame(dataset['train'])

# %% [Domain Analysis Features]
class URLAnalyzer:
    def __init__(self):
        self.trusted_certs = {'geotrust', 'godaddy', 'network solutions', 'thawte', 'comodo', 'digicert', 'verisign'}
    
    def extract_features(self, url):
        features = {}
        parsed = urlparse(url)
        ext = tldextract.extract(url)
        
        # Basic URL features
        features['url_length'] = len(url)
        features['uses_ip'] = self._is_ip_address(parsed.netloc)
        features['shortened'] = any(x in parsed.netloc for x in ['bit.ly', 'tinyurl', 'goo.gl'])
        features['has_at'] = '@' in url
        features['redirects'] = url.count('//') > 1
        features['hyphen_domain'] = '-' in ext.domain
        features['subdomain_count'] = ext.subdomain.count('.') + 1 if ext.subdomain else 0
        features['https'] = parsed.scheme == 'https'
        features['port'] = parsed.port if parsed.port else 443 if features['https'] else 80
        
        # Advanced features
        features['cert_age'] = self._get_cert_age(url)
        features['domain_age'] = self._get_domain_age(ext.registered_domain)
        features['ssl_trusted'] = self._check_ssl_issuer(url)
        
        return features
    
    def generate_reasons(self, features):
        reasons = []
        
        # URL Structure Analysis
        if features['uses_ip']:
            reasons.append("Uses IP address instead of domain name")
        if features['url_length'] >= 54:
            reasons.append(f"Long URL ({features['url_length']} chars) hiding suspicious parts")
        if features['shortened']:
            reasons.append("URL shortening service detected")
        if features['has_at']:
            reasons.append("Contains '@' symbol for redirection")
            
        # Domain Characteristics
        if features['hyphen_domain']:
            reasons.append("Hyphen in domain name")
        if features['subdomain_count'] > 2:
            reasons.append(f"Multiple subdomains ({features['subdomain_count']})")
            
        # Security Features
        if not features['https']:
            reasons.append("No HTTPS encryption")
        elif not features['ssl_trusted']:
            reasons.append("Untrusted SSL certificate issuer")
        if features['port'] not in [80, 443]:
            reasons.append(f"Non-standard port ({features['port']}) used")
            
        # Temporal Features
        if features['domain_age'] < 365:
            reasons.append(f"New domain ({features['domain_age']} days old)")
        if features['cert_age'] < 365:
            reasons.append(f"New SSL certificate ({features['cert_age']} days old)")
            
        return reasons
    
    def _is_ip_address(self, netloc):
        try:
            ipaddress.ip_address(netloc)
            return True
        except:
            return False
    
    def _get_cert_age(self, url):
        # Placeholder for actual certificate check
        return np.random.randint(1, 365*3)  # Simulated certificate age
    
    def _get_domain_age(self, domain):
        # Placeholder for WHOIS lookup
        return np.random.randint(1, 365*5)  # Simulated domain age
    
    def _check_ssl_issuer(self, url):
        # Placeholder for SSL issuer check
        return np.random.choice([True, False])

# %% [Data Preprocessing]
analyzer = URLAnalyzer()

def process_data(row):
    features = analyzer.extract_features(row['url'])
    reasons = analyzer.generate_reasons(features)
    label = 'Phishing' if row['label'] == 1 else 'Legitimate'
    
    return {
        'input_text': f"Analyze URL: {row['url']}",
        'output_text': f"Verdict: {label}. Reasons: {'; '.join(reasons) if reasons else 'No suspicious indicators found'}",
        'features': str(features),
        'label': row['label']
    }

processed_df = df.apply(process_data, axis=1, result_type='expand')

# %% [Model Setup]
model_name = "gpt2-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)
model.config.pad_token_id = tokenizer.eos_token_id

# %% [Tokenization]
def tokenize_function(examples):
    full_text = [f"{inp}\n{out}" for inp, out in zip(examples['input_text'], examples['output_text'])]
    tokenized = tokenizer(
        full_text,
        max_length=256,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    return {
        'input_ids': tokenized['input_ids'],
        'attention_mask': tokenized['attention_mask'],
        'labels': tokenized['input_ids'].clone()
    }

dataset = Dataset.from_pandas(processed_df)
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# %% [Training Configuration]
training_args = TrainingArguments(
    output_dir="./phishing-detector",
    evaluation_strategy="steps",
    eval_steps=1000,
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=500,
    fp16=True,
    gradient_accumulation_steps=2,
    save_strategy="steps",
    save_steps=2000,
    load_best_model_at_end=True,
    prediction_loss_only=True
)

# %% [Training]
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset
)

trainer.train()
model.save_pretrained("./phishing-detector")
tokenizer.save_pretrained("./phishing-detector")

# %% [Inference with Confidence]
class PhishingDetector:
    def __init__(self):
        self.model = AutoModelForCausalLM.from_pretrained("./phishing-detector")
        self.tokenizer = AutoTokenizer.from_pretrained("./phishing-detector")
        self.pipe = pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            device=0 if torch.cuda.is_available() else -1
        )
        
    def analyze(self, url):
        prompt = f"Analyze URL: {url}\n"
        result = self.pipe(
            prompt,
            max_length=256,
            temperature=0.7,
            top_p=0.9,
            num_return_sequences=1,
            repetition_penalty=1.2
        )[0]['generated_text']
        
        # Parse output
        verdict = "Phishing" if "Phishing" in result else "Legitimate"
        reasons = self._extract_reasons(result)
        confidence = self._calculate_confidence(reasons)
        
        return {
            "url": url,
            "verdict": verdict,
            "confidence": confidence,
            "reasons": reasons
        }
    
    def _extract_reasons(self, text):
        try:
            return text.split("Reasons:")[1].strip().split('; ')
        except:
            return ["Unable to parse reasons"]
    
    def _calculate_confidence(self, reasons):
        base = len(reasons) * 0.15
        return min(0.99, max(0.5, base))

# %% [Evaluation]
detector = PhishingDetector()

test_urls = [
    "https://www.paypal.com",
    "http://192.168.1.1/login",
    "https://bit.ly/3xYmPa9",
    "http://secure-facebook-login.xyz",
    "https://www.google.com"
]

for url in test_urls:
    result = detector.analyze(url)
    print(f"""
🔍 Security Analysis:
URL: {result['url']}
Verdict: {result['verdict']} (Confidence: {result['confidence']:.0%})
Reasons: {', '.join(result['reasons'])}
{'🛡️ Legitimate' if result['verdict'] == 'Legitimate' else '⚠️ Phishing Alert'}
--------------------------------------------------""")

  error: subprocess-exited-with-error
  
  × Building wheel for pyarrow (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [784 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build/lib.macosx-10.9-universal2-cpython-310/pyarrow
      copying pyarrow/orc.py -> build/lib.macosx-10.9-universal2-cpython-310/pyarrow
      copying pyarrow/conftest.py -> build/lib.macosx-10.9-universal2-cpython-310/pyarrow
      copying pyarrow/_generated_version.py -> build/lib.macosx-10.9-universal2-cpython-310/pyarrow
      copying pyarrow/benchmark.py -> build/lib.macosx-10.9-universal2-cpython-310/pyarrow
      copying pyarrow/_compute_docstrings.py -> build/lib.macosx-10.9-universal2-cpython-310/pyarrow
      copying pyarrow/ipc.py -> build/lib.macosx-10.9-universal2-cpython-310/pyarrow
      copying pyarrow/util.py -> build/lib.macosx-10.9-universal2-cpython-310/pyarrow
      copying pyarrow/flight.py -> build/lib.macosx-10.9-universa

ModuleNotFoundError: No module named 'tldextract'